In [ ]:
#required modules

import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
import skimage
from skimage import filters
from skimage.morphology import disk
from skimage.filters import threshold_mean, threshold_isodata, threshold_minimum, threshold_otsu, threshold_triangle, threshold_li
#if you need to implement something about image processing, first check if there is already a function available in
#the 


In [ ]:
#create an image, which is just a 2d matrix.
image_size=1024
image=np.zeros((image_size, image_size))
image=np.random.random((image_size, image_size))

#~in case we want an all zeros for backgorund, just use the 0.
#if we want a random values disributed all along, use random and the type of distirbution we want
#although it is better to add it at the end of the process to be able to control better the levels of noise with respect
#the actual signal of the peaks, you can even use the following to directly apply the poissonian noise to the default image:
image_poisson=np.random.poisson((image_size, image_size))


In [ ]:
#to plot the image with pyplot
plt.imshow(image)
plt.show()

In [ ]:
#important to know how to index the image:
#a 2D numpy array needs two indices to be referenced, the values of which lay between 0 and image_size-1, being 0 the 
#smallest possible and image_size-1 the largest
index_y=1
index_x=1
index_y1=1
index_y2=30
index_x1=1
index_x2=30
certain_image_element=image[index_y, index_x]
#always the first index that is presented between [] is the y index, vertical, direction, image rows,
#while the second is referencing to x, horizontal directions, or columns in the image
#the index image[0,0] goes to the top left pixel, and the image[image_size-1, image_size-1], is the bottom right one

top_left_pixel=image[0,0]
bottom_right_pixel=image[image_size-1, image_size-1]
#crop the image
cropped_image=image[index_y1:index_y2, index_x1:index_x2]



In [ ]:
#in the case of dealing with images and their FFT, we need to have them properly calibrated
#the calibration can be done via pixel size or by FOV and total pixels (which is eventually defining the pixel size)
FOV=100  #nm
pixel_size=FOV/image_size
#then every image needs to have a label on which calibration it has:
#to do so, we use the hyperspy package as it has all the functionalities we need 

hyperspy_2D_signal=hs.signals.Signal2D(image)
#we modify the metadata where the calibration is stored
hyperspy_2D_signal.axes_manager.set_signal_dimension(2)
hyperspy_2D_signal.axes_manager[0].name='x'
hyperspy_2D_signal.axes_manager[1].name='y'
hyperspy_2D_signal.axes_manager['x'].scale=pixel_size
hyperspy_2D_signal.axes_manager['y'].scale=pixel_size
hyperspy_2D_signal.axes_manager['x'].units='nm'
hyperspy_2D_signal.axes_manager['y'].units='nm'

#then we can get our FFT of the iamge calibrated automaticallly by just applying the FFT method to the numpy array, 
#that by now it is a hyperspy singal
fft_shifted = hyperspy_2D_signal.fft(shift=True)   #the shift makes that the FFT spots appear in the central region of the image
FFT_calibration=fft_shifted.axes_manager['x'].scale
#you can go back to numpy array in case you want, from the hyperspy singal:
image_again=np.asarray(hyperspy_2D_signal)
#then, if you want an image to be calibrated given a FOV/pixel size, you can directly get the calibration of the corresponding FFT
#and use these values to build the image based on the distances and reciprocal distnaces obtained by the simulation software
# (you have this functions in the kineDP notebook!)


#the same can be obtained with numpy arrays, but we lose the calibration information:
fft_numpy = np.abs(np.fft.fftshift(np.log(np.fft.fft2(image))))

In [ ]:
#one very intersting property of numpy arrays is the broadcasting and the bolean or mask indexing
#broadcast allows for doing some operations that you would expect to create a loop but just by using single operations
#e.g. 
vector=np.array([1,2,3,4,5,6,7,8,9,10])
#I want to create a vector whose number is vector+1. Two ways of doing so:
#1) very slow:
vector1=[]
for value in vector:
    new_val=value+1
    vector1.append(new_val)
vector1=np.asarray(vector1)


In [ ]:
#2) using the broadcasting property: faster and easier:
vector1=vector+1 
#it will still consider properly the dimensions of the vector and matrixes and operate accordingly


In [ ]:
#masking is even more interesting, as allows for very quick boolean indexing
#e.g. from our matrix of zeros (background) and ones (signal), we want to get the coordiantes of the ones. Two ways:
#1) very slow, looping throught the pixels:
indexes=[]
for index_row, row in enumerate(image):
    for index_col, column in enumerate(row):
        if column==1:
            indexes.append([index_row,index_col])

In [ ]:
#2 using the boolean masking of numpy arrays:
indexes=image==1
#this will not give you the list of indexes, but a matrix with True where image==1 and False otherwise. Then, pixelwise multiplications
#will only affect these values if you want only to interact with them
#this can be a really effective way of applying a threshold:
# take only the pixels of the image whose FFT is more intense than 0.5 --> only the spots
FFT_threshold=0.5
image[fft_numpy>FFT_threshold]=1
image[fft_numpy<=FFT_threshold]=0


In [ ]:
#3 or even with where:
indexes=np.where(image==1)

#conclusion: think twice before doing a loop, because they are fast in c but awful in python!!!

In [ ]:
#when we have our image, it will be important to have a dataset that is comparable with itself
#this means all the images have values of intensity between 0 and 1.
#to do so, just normalise them:


def normalize_image(image):
    """
    Normalizes the provided image from 0 to 1

    Parameters
    ----------
    image : np.array object
        Image to be normalized

    Returns
    -------
    image : np.array object
        Image normalized from 0 to 1
    """
    return (image - np.amin(image)) / (np.amax(image) - np.amin(image))

image_norm=normalize_image(image)

In [ ]:
#to plot the image with pyplot
plt.imshow(image_norm)
plt.show()
#and visualise the histogram. The histogram is a very nice and quick way of characterising the FFT, as all the FFTs of 
#atomically resolved image have more or less the same shape of histogram:
plt.hist(image_norm.ravel(),256,[np.min(np.array([image_norm])),np.max(np.array([image_norm]))])
plt.show()

In [ ]:
#plot the images with different colours:

fig, axes = plt.subplots(ncols=2, figsize=(8, 4))
# Viridis is assigned by default even if the cmap argument is not specified
im_handle = axes[0].imshow(image_norm)
axes[0].set_title('cmap="viridis"')
# Adding a colorbar:
cbar = plt.colorbar(im_handle, ax=axes[0], orientation='vertical',
fraction=0.046, pad=0.04, use_gridspec=True)

im_handle = axes[1].imshow(image_norm, cmap='jet')
axes[1].set_title('cmap="jet"')
# Adding a colorbar:
cbar = plt.colorbar(im_handle, ax=axes[1], orientation='vertical',
fraction=0.046, pad=0.04, use_gridspec=True)
fig.tight_layout()

In [ ]:
# image filters:
cropped_image = image[:128, :128]

plt.imshow(cropped_image)
plt.title('Original image')
plt.show()
# Local mean filter
loc_mean = filters.rank.mean(cropped_image, disk(4))

# Median filter:
median_filtered = filters.median(cropped_image, disk(5))

# Gaussian Filter
gaussian_filtered = filters.gaussian(cropped_image, sigma=1.875)

# Bilateral mean
noisy_image = skimage.img_as_ubyte(cropped_image)
bilat_filtered = filters.rank.mean_bilateral(noisy_image.astype(np.uint16), disk(10), s0=5, s1=5)

# Self-tuned Wiener
disk_size = int(1)
psf = np.ones((disk_size, disk_size)) / disk_size**2
self_tuned, _ = skimage.restoration.unsupervised_wiener(cropped_image, psf)

# Wiener deconvolution
disk_size = int(10)
psf = np.ones((disk_size, disk_size)) / disk_size**2
wiener_filtered = skimage.restoration.wiener(cropped_image, psf, 50)

# Visualizing results here:
fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(12, 8),
                         sharex=True, sharey=True)
for axis, img, title in zip(axes.flat, 
                            [loc_mean, median_filtered, gaussian_filtered, 
                             self_tuned, bilat_filtered, wiener_filtered],
                            ['Local mean', 'Median', 'Gaussian', 
                             'Self-tuned Restoration', 'Mean Bilatereral', 'Wiener']):
    axis.imshow(img, cmap='gray')
    axis.set_title(title)
fig.suptitle('Filters', fontsize=18, y=1.03)
fig.tight_layout()

In [ ]:
#edge detection:
#probably not being useful in our particular case, but just in case
fig, axes = plt.subplots(figsize=(8, 8), nrows=2, ncols=2, 
                         sharex=True, sharey=True)
for axis, filt_alg, title in zip(axes.flat, 
                                 [filters.sobel, filters.scharr,
                                  filters.prewitt, filters.roberts],
                                 ['Sobel', 'Scharr', 'Prewitt', 'Roberts']):
    axis.imshow(filt_alg(cropped_image), cmap='gray')
    axis.set_title(title, fontsize=16)
fig.tight_layout()

In [ ]:
#Final comment: if you find any function that is implemented in one package, and not in another, use it and then you must 
#about only care about data type conversions, which for the main packages, skimage, cv2, hyprespy and numpy they are 
#always supported!